<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/StableLMJ20231026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# nvidia-smiを使ってVRAMのサイズを取得
vram_str = !nvidia-smi --query-gpu=memory.total --format=csv,noheader,nounits
print(vram_str)

from google.colab import runtime
# nvidia-smiが正常に動作しない場合のエラーハンドリング
if "command not found" in vram_str[0]:
    print("GPUが有効になっていないか、nvidia-smiが利用できません。")
    print("セッションを削除します。")
    exit();
else:
    try:
        vram_size = int(vram_str[0]) / 1024  # MiBからGBに変換

        # VRAMが16GB未満の場合、メッセージを表示してセッションをリセット
        if vram_size < 16:
            print("VRAMが16GB未満です。セッションをリセットしてください。")
            try:
              runtime.unassign()
            except:
              runtime.quit()
    except ValueError:
        print("VRAMのサイズを取得する際にエラーが発生しました。")
        print("セッションを再起動します。")
        exit()

# 最後に現在のVRAMを表示
print(f"現在のVRAM: {vram_size:.2f} GB")


['16384']
現在のVRAM: 16.00 GB


In [3]:
!pip install transformers sentencepiece gradio ftfy 'accelerate>=0.12.0' 'bitsandbytes>=0.31.5' einops
#!pip install accelerate bitsandbytes
#!pip install transformers sentencepiece
#!pip install accelerate bitsandbytes
#!pip einops
#!pip install gradio ftfy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 34.7 MB/s eta

In [4]:
# @title Login HuggingFace
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [5]:
# @title cuda or CPU
import torch
from transformers import AutoTokenizer, LlamaTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

device: cuda


In [26]:
# !accelerate config
# !accelerate test


Running:  accelerate-launch /usr/local/lib/python3.10/dist-packages/accelerate/test_utils/scripts/test_script.py
stderr: The following values were not passed to `accelerate launch` and had defaults used instead:
stderr: 	`--num_cpu_threads_per_process` was set to `4` to improve out-of-box performance when training on CPUs
stderr: To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
stdout: **Initialization**
stdout: Testing, testing. 1, 2, 3.
stdout: Distributed environment: NO
stdout: Num processes: 1
stdout: Process index: 0
stdout: Local process index: 0
stdout: Device: cpu
stdout: 
stdout: Mixed precision type: no
stdout: 
stdout: 
stdout: **Test process execution**
stdout: 
stdout: **Test split between processes as a list**
stdout: 
stdout: **Test split between processes as a dict**
stdout: 
stdout: **Test split between processes as a tensor**
stdout: 
stdout: **Test random number generator synchronization**
stdout: All rng are p

In [12]:
# @title Load Model
model_id = "stabilityai/japanese-stablelm-instruct-gamma-7b" # @param ["stabilityai/japanese-stablelm-instruct-alpha-7b", "stabilityai/japanese-stablelm-instruct-gamma-7b", "stabilityai/japanese-stablelm-3b-4e1t-instruct"]

# model_kwargs = {"trust_remote_code": True, "device_map": "auto", "low_cpu_mem_usage": True, "torch_dtype": "auto"}
model_kwargs = {"trust_remote_code": True, "device_map": {"",0} , "load_in_8bit": True }

if "alpha" in model_id:
  tokenizer = LlamaTokenizer.from_pretrained("novelai/nerdstash-tokenizer-v1", additional_special_tokens=['▁▁'])
else:
  tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False)

# model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)
model = AutoModelForCausalLM.from_pretrained(model_id,
          trust_remote_code=True,load_in_8bit=True, device_map="auto",variant="int8")
model = model.eval().to(device)

OSError: ignored

In [ ]:
# @title **Do the Run!**
# @markdown Try Japanese Stable LM in chat-like UI.
# @markdown <br>**Remark:** this is single-turn inference, i.e., previous contexts are ignored.
import gradio as gr


def build_prompt(user_query, inputs="", sep="\n\n### "):
    sys_msg = "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。"
    p = sys_msg
    roles = ["指示", "応答"]
    msgs = [": \n" + user_query, ": \n"]
    if inputs:
        roles.insert(1, "入力")
        msgs.insert(1, ": \n" + inputs)
    for role, msg in zip(roles, msgs):
        p += sep + role + msg
    return p

@torch.no_grad()
def base_inference_func(prompt, max_new_tokens=128, top_p=0.95, repetition_penalty=1.):
  # print(f"PROMPT:\n{prompt}")
  input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
  output_ids = model.generate(
      input_ids.to(model.device),
      do_sample=True,
      max_new_tokens=max_new_tokens,
      top_p=top_p,
      temperature=1,
      repetition_penalty=repetition_penalty,
  )

  generated = tokenizer.decode(output_ids.tolist()[0][input_ids.size(1):], skip_special_tokens=True).strip()
  # print(f"generated: {generated}")
  return generated


def inference_func(message, chat_history, additional_prompt, max_new_tokens=128, top_p=0.95, repetition_penalty=1.):
  # Infer with prompt without any additional input
  user_inputs = {
      "user_query": message,
      "inputs": additional_prompt,
  }
  prompt = build_prompt(**user_inputs)
  generated = base_inference_func(prompt, max_new_tokens, top_p, repetition_penalty)
  chat_history.append((message, generated))
  return "", chat_history


with gr.Blocks() as demo:
  with gr.Accordion("Configs", open=False):
      if "instruct" in model_id:
        instruction = gr.Textbox(label="instruction",)
      max_new_tokens = gr.Number(value=128, label="max_new_tokens", precision=0)
      top_p = gr.Slider(0.0, 1.0, value=0.95, step=0.01, label="top_p")
      repetition_penalty = gr.Slider(0.0, 5.0, value=1.1, step=0.1, label="repetition_penalty")

  if "instruct" in model_id:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    msg.submit(inference_func, [msg, chatbot, instruction, max_new_tokens, top_p, repetition_penalty], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)
  else:
    with gr.Row():
      with gr.Column():
        prompt = gr.Textbox(label="prompt")
        button = gr.Button(label="submit")
      with gr.Column():
        out = gr.Textbox(label="generated")
    button.click(base_inference_func, [prompt, max_new_tokens, top_p, repetition_penalty], out)

if __name__ == "__main__":
    demo.launch(debug=True, share=True, show_error=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://76a416e4879d593e49.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# @markdown ↓`japanese-stablelm-instruct-gamma-7b` が最新のモデルです
model_id = "stabilityai/japanese-stablelm-3b-4e1t-instruct" # @param ["stabilityai/japanese-stablelm-instruct-alpha-7b", "stabilityai/japanese-stablelm-instruct-gamma-7b", "stabilityai/japanese-stablelm-3b-4e1t-instruct"]

# tokenizer = AutoTokenizer.from_pretrained("stabilityai/japanese-stablelm-instruct-gamma-7b")
tokenizer = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype="auto", device_map="auto")
if torch.cuda.is_available():
    model = model.to("cuda")
model.eval()


The repository for stabilityai/japanese-stablelm-3b-4e1t-instruct contains custom code which must be executed to correctlyload the model. You can inspect the repository content at https://hf.co/stabilityai/japanese-stablelm-3b-4e1t-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


A new version of the following files was downloaded from https://huggingface.co/stabilityai/japanese-stablelm-3b-4e1t-instruct:
- configuration_stablelm_epoch.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


The repository for stabilityai/japanese-stablelm-3b-4e1t-instruct contains custom code which must be executed to correctlyload the model. You can inspect the repository content at https://hf.co/stabilityai/japanese-stablelm-3b-4e1t-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


A new version of the following files was downloaded from https://huggingface.co/stabilityai/japanese-stablelm-3b-4e1t-instruct:
- modeling_stablelm_epoch.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


RuntimeError: ignored

In [ ]:
# @title Load model
import torch
from transformers import LlamaTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

model_id = "stabilityai/japanese-stablelm-instruct-gamma-7b" # @param ["stabilityai/japanese-stablelm-instruct-gamma-7b", "stabilityai/japanese-stablelm-3b-4e1t-instruct"]
#load_in = "int8" # @param ["fp32", "fp16", "int8"]
# @markdown If you use Colab free plan, please make sure to set `load_in` to `int8`.

model_kwargs = {"trust_remote_code": True, "device_map": "auto", "low_cpu_mem_usage": True}
if load_in == "fp16":
  model_kwargs["variant"] = "fp16"
  mddel_kwargs["torch_dtype"] = torch.float16
elif load_in == "int8":
  model_kwargs["variant"] = "int8"
  model_kwargs["load_in_8bit"] = True
tokenizer = LlamaTokenizer.from_pretrained("novelai/nerdstash-tokenizer-v1", additional_special_tokens=['▁▁'])
model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)
if load_in != "int8":
  model.to(device)
model.eval()

In [ ]:


def build_prompt(user_query, inputs="", sep="\n\n### "):
    sys_msg = "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。"
    p = sys_msg
    roles = ["指示", "応答"]
    msgs = [": \n" + user_query, ": \n"]
    if inputs:
        roles.insert(1, "入力")
        msgs.insert(1, ": \n" + inputs)
    for role, msg in zip(roles, msgs):
        p += sep + role + msg
    return p

# Infer with prompt without any additional input
user_inputs = {
    "user_query": "与えられたことわざの意味を小学生でも分かるように教えてください。",
    "inputs": "情けは人のためならず"
}
prompt = build_prompt(**user_inputs)

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=256,
    temperature=1,
    top_p=0.95,
    do_sample=True,
)

out = tokenizer.decode(tokens[0][input_ids.shape[1]:], skip_special_tokens=True).strip()
print(out)

OutOfMemoryError: ignored